In [ ]:
import wandb
from fastai.vision.all import *

In [ ]:
path = untar_data(URLs.CAMVID)
codes = np.loadtxt(path/'codes.txt', dtype=str)
fnames = get_image_files(path/"images")
class_labels = {k: v for k, v in enumerate(codes)}

In [ ]:
PROJECT="CamVid"
ENTITY="av-demo"

In [ ]:
with wandb.init(
    project=PROJECT,
    name="upload_camvid",
    entity=ENTITY,
    job_type="upload",
):
    artifact = wandb.Artifact(
        'camvid-dataset',
        type='dataset',
        metadata={
            "url": URLs.CAMVID,
            "class_labels": class_labels
        },
        description="The Cambridge-driving Labeled Video Database (CamVid) is the first collection of videos with object class semantic labels, complete with metadata. The database provides ground truth labels that associate each pixel with one of 32 semantic classes."
    )
    artifact.add_dir(path)
    wandb.log_artifact(artifact)

In [ ]:
def label_func(fn):
    return fn.parent.parent/"labels"/f"{fn.stem}_P{fn.suffix}"

In [ ]:
def log_dataset():
    # image_files = sorted(glob.glob(os.path.join(str(path), "images/*.png")))
    # label_files = sorted(glob.glob(os.path.join(str(path), "labels/*.png")))
    with wandb.init(
        project=PROJECT,
        name="visualize_camvid",
        entity=ENTITY,
        job_type="process_dataset"
    ):
        artifact = wandb.use_artifact('av-demo/CamVid/camvid-dataset:v0', type='dataset')
        artifact_dir = artifact.download()
        
        image_files = get_image_files(Path(artifact_dir)/"images")
        
        table_data = []
        
        print("Creating Table...")
        for image_file in tqdm(image_files):
            image = np.array(Image.open(image_file))
            mask_data = np.array(Image.open(label_func(image_file)))
            table_data.append([
                str(image_file.name),
                wandb.Image(image),
                wandb.Image(image, masks={
                    "predictions": {
                        "mask_data": mask_data,
                        "class_labels": class_labels
                    }
                })
            ])
        wandb.log({"CamVid_Dataset": wandb.Table(
            data=table_data, columns=["File_Name", "Images", "Segmentation_Masks"]
        )})

In [ ]:
log_dataset()